## Урок 3. Парсинг HTML

1) Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта superjob.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе: 
- Наименование вакансии, 
- Предлагаемую зарплату,
- Ссылку на саму вакансию

In [256]:
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
import requests
import re

def get_link(main_link, add_link, key_word):
    link = main_link + add_link + key_word
    return link

def get_html(link, headers = {}):
    if link[0:4] == 'http':
        html = requests.get(link, headers=headers).text
    else:
        with open(link, 'rb') as f:
            html = f.read()
    return html

def parsing_html(link, headers = {}):
    html =  get_html(link, headers=headers)
    parsed_html = bs(html, 'html.parser')
    return parsed_html

def find_class(html, class_):
    found_class = html.findAll(class_=class_)
    return found_class

def find_item(html, class_, href=False):
    found_item = html.find(class_=class_, href=href)
    return found_item

def get_vacancy(html, class_):
    found_class = find_class(html, class_[0])
    vacancy_info = []
    for i in found_class:
        class_dict = {}
        vacancy = find_item(i, class_[1], href=True)
        class_dict['vacancy'] = vacancy.get_text()
        salary = find_item(i, class_[2])
        salary = salary.get_text() if salary else salary
        print(salary)
        salary = re.findall('([\d]+)[\s]*([\d]+)', salary) if salary else '0'
        class_dict['salary'] = [''.join(i) for i in salary] if len(salary) else ['0']
        link = vacancy['href']
        link = link if link[0:4]=='http' else main_link + link
        link = re.findall('http[s]?:\/\/[^\?]*', link)
        class_dict['link'] = link[0] if len(link) else ''
        vacancy_info.append(class_dict)
    return vacancy_info

#### hh.ru - работает GET запрос через requests только при передаче заголовка с параметром UserAgent (можно с пустым).

In [257]:
main_link = 'https://hh.ru'
add_link = '/search/vacancy'
key_word = '?text=Программист Разработчик'
class_ = ['vacancy-serp-item', 'bloko-link HH-LinkModifier', 'vacancy-serp-item__compensation']

headers = {'User-Agent': UserAgent().chrome}
link = get_link(main_link, add_link, key_word)
# link = 'hh.html'
parsed_html = parsing_html(link, headers)
vacancy_hh = get_vacancy(parsed_html, class_)
vacancy_hh

80 000-150 000 руб.
10 000-100 000 KGS
80 000-120 000 руб.
80 000-120 000 руб.
55 000-70 000 руб.
None
от 180 000 руб.
150 000-190 000 руб.
None
130 000-160 000 руб.
130 000-160 000 руб.
None
None
None
от 150 000 руб.
180 000-200 000 руб.
180 000-200 000 руб.
от 900 USD
4 000-6 000 USD
None


[{'vacancy': 'Программист Javascript',
  'salary': ['80000', '150000'],
  'link': 'https://hh.ru/vacancy/32116925'},
 {'vacancy': 'Программист-разработчик',
  'salary': ['10000', '100000'],
  'link': 'https://hh.ru/vacancy/32524602'},
 {'vacancy': 'Программист С++ (графика)',
  'salary': ['80000', '120000'],
  'link': 'https://hh.ru/vacancy/32543357'},
 {'vacancy': 'Программист iOS (Swift)',
  'salary': ['80000', '120000'],
  'link': 'https://hh.ru/vacancy/31399856'},
 {'vacancy': 'Junior PHP-разработчик',
  'salary': ['55000', '70000'],
  'link': 'https://hh.ru/vacancy/33189536'},
 {'vacancy': 'Frontend-разработчик',
  'salary': ['0'],
  'link': 'https://hh.ru/vacancy/32901208'},
 {'vacancy': 'Ведущий программист 1C / Ведущий разработчик 1С',
  'salary': ['180000'],
  'link': 'https://hh.ru/vacancy/33192429'},
 {'vacancy': 'C# back-end разработчик',
  'salary': ['150000', '190000'],
  'link': 'https://hh.ru/vacancy/29832222'},
 {'vacancy': 'Front-end разработчик',
  'salary': ['0'],
 

#### superjob.ru - работает GET запрос через requests без передачи заголовка

In [258]:
main_link = 'https://superjob.ru'
add_link = '/vacancy/search/'
key_word = '?keywords=Программист|Разработчик'
class_ = ['_3syPg', '_1QIBo', '_2Wp8I']

link = get_link(main_link, add_link, key_word)
# link = 'sj.html'
parsed_html = parsing_html(link)
vacancy_sj = get_vacancy(parsed_html, class_)
vacancy_sj

По договорённости
от 140 000 ₽
от 80 000 ₽
от 100 000 ₽
По договорённости
от 80 000 ₽
от 60 000 ₽
90 000 ₽
По договорённости
от 120 000 ₽
150 000 — 180 000 ₽
По договорённости
54 000 — 64 000 ₽
По договорённости
от 170 000 ₽
По договорённости
70 000 — 120 000 ₽
100 000 — 120 000 ₽
По договорённости
По договорённости
184 000 — 212 000 ₽
от 150 000 ₽
55 000 ₽
100 000 ₽
По договорённости
150 000 ₽
49 000 — 60 000 ₽
150 000 ₽
от 100 000 ₽
По договорённости


[{'vacancy': 'Web-программист, Web-разработчик',
  'salary': ['0'],
  'link': 'https://superjob.ru/vakansii/web-programmist-32266222.html'},
 {'vacancy': 'Программист / Разработчик 1С 8.3 УПП',
  'salary': ['140000'],
  'link': 'https://superjob.ru/vakansii/programmist-32418578.html'},
 {'vacancy': 'Программист / Разработчик',
  'salary': ['80000'],
  'link': 'https://superjob.ru/vakansii/programmist-32274027.html'},
 {'vacancy': 'Веб-программист, веб-разработчик',
  'salary': ['100000'],
  'link': 'https://superjob.ru/vakansii/veb-programmist-32441272.html'},
 {'vacancy': 'Программист-разработчик (.NET) на ELMA BPM',
  'salary': ['0'],
  'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-32408682.html'},
 {'vacancy': 'Программист (разработчик 1С)',
  'salary': ['80000'],
  'link': 'https://superjob.ru/vakansii/programmist-32221501.html'},
 {'vacancy': 'Программист-разработчик',
  'salary': ['60000'],
  'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-32382559

2) Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)

In [176]:
main_link = 'https://superjob.ru'
add_link = '/vacancy/search/'
key_word = '?keywords=Программист|Разработчик Python'
class_ = ['_3syPg', '_1QIBo', '_2Wp8I']

link = get_link(main_link, add_link, key_word)
# link = 'sj.html'
parsed_html = parsing_html_page(link)
vacancy_sj = get_vacancy(parsed_html, class_)
vacancy_sj

[{'vacancy': 'Аналитик-разработчик Python',
  'salary': '',
  'link': 'https://superjob.ru/vakansii/analitik-razrabotchik-python-32360537.html'},
 {'vacancy': 'Python-разработчик (backend)',
  'salary': '70000',
  'link': 'https://superjob.ru/vakansii/python-razrabotchik-32165239.html'},
 {'vacancy': 'JavaScript (веб) программист',
  'salary': '110000120000',
  'link': 'https://superjob.ru/vakansii/javascript-32434463.html'},
 {'vacancy': 'Программист 1С',
  'salary': '110000150000',
  'link': 'https://superjob.ru/vakansii/programmist-1s-32355948.html'},
 {'vacancy': 'Программист 1С',
  'salary': '',
  'link': 'https://superjob.ru/vakansii/programmist-1s-32174069.html'},
 {'vacancy': 'Программист C++, Qt, QML',
  'salary': '',
  'link': 'https://superjob.ru/vakansii/programmist-c-32405114.html'},
 {'vacancy': 'Программист АБС ЦФТ',
  'salary': '',
  'link': 'https://superjob.ru/vakansii/programmist-abs-cft-32263158.html'},
 {'vacancy': 'Программист javascript (телекоммуникационные услу